## Convertire i file dicom in file png cercando di mantenere coerente la scala dei grigi.


Il procedimento è molto simile in entrambi i casi, anche perché matplotlib sfrutta PIL per visualizzare le immagini.
In sostanza si apre il file dicom con pydicom da cui si estrae il "pixel_array" convertendo i valori al suo interno in float, sfruttando numpy per evitare underflow o overflow durante la conversione. 
Dopo aver usato apply_modality_lut() e apply_voi_lut() è possibile eseguire la conversione riducendo il range della scala dei grigi da 16 bit a 8 bit e salvare l'immagine in png.

In [21]:
#Convertire file DICOM in PNG con matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
from pydicom.pixel_data_handlers.util import apply_modality_lut

ds= pydicom.dcmread('img2.dicom')

p_arr = ds.pixel_array.astype(float) 
#array di pixel estratti dal file dicom e astype(float) per evitare problemi di overflow o underfolow 

mod = apply_modality_lut(p_arr, ds) 
# applicazione della modality look up table

voi = apply_voi_lut(mod, ds) 
# applicazione della voi look up table

s_arr = (np.maximum(voi, 0) / voi.max()) * 255.0 
#la scala dei grigi usata nell'immagine passa da 16 a 8 bit

s_arr = np.uint8(s_arr) #conversione in Unsigned Int

"""#immagine prima del rescaling
print(p_arr.dtype)
print(p_arr.shape)
print(p_arr)

#immagine dopo il rescaling

print(s_arr.dtype)
print(s_arr.shape)
print(s_arr)
"""

plt.imshow(scaled_image, cmap = plt.cm.bone, vmin = 0, vmax = 255)
#plt.imsave("matplotlib.png", scaled_image, cmap = plt.cm.bone, vmin = 0, vmax = 255) 
# usando "gray" invece di "bone" si rende produce un file.png più leggero ma, a mio avviso, meno leggibile

uint16
(3072, 3072)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 3 ... 3 0 0]
 ...
 [0 0 3 ... 3 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
uint8
(3072, 3072)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [22]:
#Convertire file DICOM in PNG con PIL

import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
from pydicom.pixel_data_handlers.util import apply_modality_lut
from PIL import Image

ds = pydicom.dcmread('path')

p_arr = ds.pixel_array.astype(float)
#array di pixel estratti dal file dicom e astype(float) per evitare problemi di overflow o underfolow 

mod = apply_modality_lut(p_arr, ds)

voi = apply_voi_lut(mod, ds)

s_arr = (np.maximum(voi, 0) / voi.max()) * 255.0

s_arr = np.uint8(s_arr)

#immagine prima del rescaling
print(p_arr.dtype)
print(p_arr.shape)
print(p_arr)

#immagine dopo il rescaling

print(s_arr.dtype)
print(s_arr.shape)
print(s_arr)


final_image = Image.fromarray(s_arr)
final_image.show()
#final_image.save('PIL.png')


FileNotFoundError: [Errno 2] No such file or directory: 'path'

La grossa differenza tra le due "strategie" è la dimensione dei file genrati. Con PIL infatti si possono genera file più leggeri. Con matplotlib i file generati sono più pesanti ma allo stesso tempo simili a quelli presenti nella documentazione che ho ricevuto https://www.kaggle.com/kmader/x-ray-patient-scan-registration.

Quale conviene seguire?

Inoltre in molte immagini appartenenti al dataset sono presenti parti del corpo non necessariamente inerenti al nostro scopo. Devo tagliare le immagini in modo da analizzare soltanto il torace?